# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [1]:
from collections import defaultdict
from itertools import product
from bs4 import BeautifulSoup
import requests
import numpy as np
from numpy.random import choice
import re
import numpy.random as random
import math

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [2]:

def dna_to_rna(s):
    "Transforms the DNA string s into an RNA string."
    dic = dict(A="A", C="C", G="G", T="U")
    return "".join(dic[l] for l in s)
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution

fill in 

### Discussion

fill in 

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [3]:
def get_dict():
    """Reads the hash table for the codon, amino acid pair and outputs the corresponding pair in a dictionary where the codon is the key"""
    
    url = 'https://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N'

    r = requests.get(url)

    html = r.text

    soup = BeautifulSoup(html)
    soup = soup.get_text()
    dict_codon_to_amino = {}

    codons_amino = re.findall(r"[A-Z]{3}\s[A-Z\*]\s\d", soup)
    if codons_amino != []:
        for codon_amino in codons_amino:
            codon, amino, _ = codon_amino.split()
            dict_codon_to_amino[codon] = amino

    return dict_codon_to_amino


### Idea of solution

fill in 

### Discussion

fill in 

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [4]:

def get_dict_list():
    """Uses the get_dict() function to create a dictionary where the keys are the amino acids and the value is a list of codons that
    correpond to this amino acid."""
    codon_to_aa = get_dict()
    aa_to_codons = {val:[] for val in codon_to_aa.values()}

    for key, val in codon_to_aa.items():
        aa_to_codons[val].append(key)
    
    return aa_to_codons
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

fill in 

### Discussion

fill in 

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [5]:

def rna_to_prot(s):
    """Converts an input DNA sequence s into a protein sequence."""
    dic = get_dict()
    if len(s) % 3 != 0:
        return
    else:
        y = re.findall(r'[A-Z]{3}', s)
        return "".join([dic[codon] for codon in y])

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

fill in 

### Discussion

fill in 

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [6]:

def get_probabability_dict():
    """
    Return the probability dicitonary with codons as keys and probabilities as values. The probabilities are taken over the space of codons that
    represent an amino acid.
    """
    dic_init = {amino:[] for amino in get_dict_list()}
    with open("table.html") as my_file:
        for line in my_file:
            patt_line = re.findall(r'([A-Z]{3,3}) ([A-Z]|\*).{11}(\(\s?\d*\))', line)
            if len(patt_line) != 4:
                continue
            else:
                for patt_tuple in patt_line:
                    amino = patt_tuple[1]
                    dic_init[amino].append((patt_tuple[0], int(patt_tuple[2].strip("() "))))
    
    dic_total = {}
    for amino in dic_init:
        total = 0
        for tuple in dic_init[amino]:
            total += tuple[1]
        dic_total[amino] = total
    
    dic_codons = {}
    for amino in dic_init:
        for tuple in dic_init[amino]:
            dic_codons[tuple[0]] = tuple[1] / dic_total[amino]
    
    return dic_codons

    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

{'UUU': 0.4641342698782968, 'UUC': 0.5358657301217032, 'UCU': 0.18758584014344437, 'UCC': 0.21795953165920925, 'UCA': 0.15051714801827132, 'UCG': 0.05439771371883908, 'AGU': 0.14960182300967595, 'AGC': 0.23993794345056002, 'UAU': 0.443338109266921, 'UAC': 0.5566618907330789, 'UGU': 0.45615733050366836, 'UGC': 0.5438426694963316, 'UUA': 0.07656764558436285, 'UUG': 0.1290578537068707, 'CUU': 0.13171591206484023, 'CUC': 0.1955768259144855, 'CUA': 0.0713801723134756, 'CUG': 0.39570159041596514, 'UAA': 0.29701911804823383, 'UGA': 0.46624296805302623, 'UAG': 0.23673791389873997, 'UGG': 1.0, 'CCU': 0.2867313296570278, 'CCC': 0.3234703981288551, 'CCA': 0.2766025276376192, 'CCG': 0.11319574457649788, 'CAU': 0.4185152128433691, 'CAC': 0.5814847871566309, 'CGU': 0.08010752804820104, 'CGC': 0.18377662978978224, 'CGA': 0.1088124833207855, 'CGG': 0.20155434006721587, 'AGA': 0.21465774794262568, 'AGG': 0.21109127083138968, 'CAA': 0.26501675921017337, 'CAG': 0.7349832407898266, 'AUU': 0.36107219301206

### Idea of solution

fill in 

### Discussion

fill in 

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [7]:
class ProteinToMaxRNA:
    prob_dic = get_probabability_dict()
    aa_to_codon = get_dict_list()
    
    def __init__(self):
        pass

    def convert(self, s):
        """Converts a protein sequence into the most likely RNA sequence to be the source of the protein sequence s."""
        rna_seq = []
        for aa in s:
            codons = ProteinToMaxRNA.aa_to_codon[aa]
            most_likely = codons[0]
            for codon in codons:
                if ProteinToMaxRNA.prob_dic[codon] > ProteinToMaxRNA.prob_dic[most_likely]:
                    most_likely = codon 
            rna_seq.append(most_likely)
        return "".join(rna_seq)

if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert('LTPIQNRA'))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

fill in 

### Discussion

fill in 

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [10]:
def random_event(dist, seed=0):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    out = random.uniform(0, 1)
    ordered_dic = list(dist.items())
    dic_prob = {}

    init = 0.0
    for tuple in ordered_dic:
        dic_prob[tuple[0]] = [init, init + tuple[1]]
        init += tuple[1]

    for event in dic_prob:
        if dic_prob[event][0] <=  out <= dic_prob[event][1]:
            return event
    #return dic_prob #next(iter(dist))

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))
    #print(random_event(distribution))

T, T, A, T, C, T, C, T, T, C, C, T, C, C, G, C, T, C, C, G, T, C, T, T, A, T, C, C, A


### Idea of solution

fill in 

### Discussion

fill in 

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [11]:
class ProteinToRandomRNA(object):
    prob_dic = get_probabability_dict()
    aa_to_codon = get_dict_list()
    
    def __init__(self):
        pass

    def convert(self, s):
        """Converts the input protein sequence s into a random RNA sequence according to the propability distribution"""
        rna_seq = []
        for aa in s:
            codons = ProteinToRandomRNA.aa_to_codon[aa]
            probabilities = [ProteinToRandomRNA.prob_dic[codon] for codon in codons]
            distribution = dict(zip(codons, probabilities))

            rna_seq.append(random_event(distribution))
        
        return "".join(rna_seq)
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

UUGACUCCUAUUCAAAAUCGGGCA


### Idea of solution

fill in 

### Discussion

fill in 

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [12]:
def sliding_window(s, k):
    """
    Returns a generator that can be iterated over all
    starting positions of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    start_index = 0
    end_index = k
    for _ in s:
        if end_index <= len(s):
            subword = s[start_index:end_index]
        else:
            break
        
        bases = ["A", "C", "T", "G"]
        dic = { base: subword.count(base) for base in bases}
        yield dic

        start_index += 1
        end_index += 1
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'A': 0, 'C': 3, 'T': 1, 'G': 0}
{'A': 0, 'C': 3, 'T': 0, 'G': 1}
{'A': 1, 'C': 2, 'T': 0, 'G': 1}
{'A': 1, 'C': 2, 'T': 0, 'G': 1}
{'A': 1, 'C': 1, 'T': 0, 'G': 2}
{'A': 1, 'C': 1, 'T': 0, 'G': 2}
{'A': 0, 'C': 2, 'T': 0, 'G': 2}
{'A': 0, 'C': 2, 'T': 0, 'G': 2}
{'A': 0, 'C': 2, 'T': 1, 'G': 1}
{'A': 0, 'C': 2, 'T': 2, 'G': 0}
{'A': 0, 'C': 1, 'T': 2, 'G': 1}
{'A': 0, 'C': 1, 'T': 2, 'G': 1}
{'A': 0, 'C': 2, 'T': 1, 'G': 1}


### Idea of solution

fill in 

### Discussion

fill in 

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [13]:
def context_list(s, k):
    """Returns dictionary where the keys are k-mers and the values are the concatenation of symbols appearing after each k-mer."""
    start = 0
    dic = {}
    while start + k <= len(s) - 1:
        kmer = s[start:start + k]
        if kmer in dic:
            dic[kmer].append(s[start + k])
        else:
            dic[kmer] = [s[start + k]]
        start += 1
    for key, val in dic.items():
        dic[key] = "".join(val)
    return dic
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

fill in

### Discussion

fill in

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [ ]:
def context_probabilities(s, k):
    """Converts frequencies from the dictionary context_list(s, k) to probabilities for each k-mer"""
    context_dic = context_list(s, k)
    dic = {}
    proteins = list("ACTG")
    for kmer in context_dic:
        dic[kmer] = {protein: (context_dic[kmer].count(protein) / len(context_dic[kmer])) for protein in proteins}
    return dic
    
if __name__ == '__main__':
    s = 'ATGATATCATCGACGATGTAG'
    k = 2
    print(context_list(s, k))
    print(context_probabilities(s, k))


{'AT': 'GACCG', 'TG': 'AT', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AG', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'GT': 'A'}
{'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4}, 'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0}, 'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0}, 'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5}, 'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0}, 'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}, 'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0}, 'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}


### Idea of solution

fill in

### Discussion

fill in

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [16]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        """Initializes the Markov Chain of kth order"""
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        left = n-self.k

        if left < 0:
            return
        
        rna_seq = ""
        for _ in range(self.k):
            protein = random_event(self.zeroth)
            rna_seq += protein
        
        for _ in range(left):
            kmer = rna_seq[-self.k:]
            protein = random_event(self.kth[kmer])
            rna_seq += protein
        
        return rna_seq


if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

CATATGATCG


### Idea of solution

fill in

### Discussion

fill in 

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [ ]:
def context_pseudo_probabilities(s, k):
    '''if k == 0:
        return dict(zip("ACGT", [0.25, 0.25, 0.25, 0.25]))
    elif k > 0:
        prod = product("ACGT", repeat=k)
        prod_strings = [str(kmer) for kmer in prod]
        context_dic = context_list(s, k)
        dic = {}
        proteins = list("ACTG")
        for kmer in prod_strings:
            if kmer in context_dic:
                dic[kmer] = {protein: (context_dic[kmer].count(protein) / len(context_dic[kmer])) for protein in proteins}
            else:
                dic[kmer] = {protein: 0.25 for protein in proteins}
        return context_dic'''
    context_prob = context_probabilities(s, k)
    all_kmers = ["".join(tuple) for tuple in product("ACGT", repeat=k)]

    for kmer in all_kmers:
        if kmer not in context_prob:
            context_prob[kmer] = dict(zip("ACGT", [0.25 for _ in range(4)]))

    return context_prob

    #return #{"": ""}
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    #zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    #print(context_pseudo_probabilities(s, k))
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
AT: {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4}
TG: {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0}
GA: {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0}
TA: {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5}
TC: {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5}
CA: {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0}
CG: {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}
AC: {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0}
GT: {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
TT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}

 TGATAGCTATGACGACGACG


### Idea of solution

fill in 

### Discussion

fill in 

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [ ]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        prob = 1.0

        for i in range(self.k):
            prob *= self.zeroth[s[i]]

        for i in range(self.k, len(s)):
            prob *= self.kth[s[i - self.k:i]][s[i]]

        return prob

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")
    print(s == 'ATGATATCATCGACGATGTAG')

Probability of sequence ATGATATCATCGACGATGTAG is 1.128747795414462e-06
True


### Idea of solution

fill in

### Discussion

fill in

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [ ]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        prob = 1.0

        for i in range(self.k):
            prob *= self.zeroth[s[i]]

        for i in range(self.k, len(s)):
            prob *= self.kth[s[i - self.k:i]][s[i]]

        return prob

    #def __init__(self, k, zeroth, kth):
    #    pass
        
    def log_probability(self, s):
        log_prob = 0.0

        for i in range(self.k):
            log_prob += math.log2(self.zeroth[s[i]])

        for i in range(self.k, len(s)):
            log_prob += math.log2(self.kth[s[i - self.k:i]][s[i]])

        return log_prob
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -19.75684539937904


### Idea of solution

fill in

### Discussion

fill in

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [ ]:
def context_probabilities(s, k):
    context_dic = context_list(s, k)
    dic = {}
    proteins = list("ACTG")
    for kmer in context_dic:
        dic[kmer] = {protein: (context_dic[kmer].count(protein) / len(context_dic[kmer])) for protein in proteins}
    return dic

def better_context_probabilities(s, k):
    start = 0
    dic = {}
    proteins = list("ACTG")
    while start + k < len(s) - 1:
        kmer = s[start:start + k]
        if kmer not in dic:
            dic[kmer] = {protein:0.0 for protein in proteins}
        dic[kmer][s[start + k]] += 1.0
        start += 1
    
    dic_probs = {}
    for kmer, freq in dic.items():
        total = sum(freq.values())
        dic_probs[kmer] = {protein: freq[protein] / total for protein in proteins}

    for kmer in product("ACGT", repeat=k):
        kmer_joint = "".join(kmer)
        if kmer_joint not in dic_probs:
            dic_probs[kmer_joint] = dict(zip("ACGT", [0.25, 0.25 , 0.25, 0.25]))
    
    return dic_probs

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

AT: {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4}
TG: {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0}
GA: {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0}
TA: {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0}
TC: {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5}
CA: {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0}
CG: {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}
AC: {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0}
GT: {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
TT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}


### Idea of solution

fill in

### Discussion

fill in

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [ ]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.k = k
        self.s = s

    def generate(self, n, seed=None):
        left = n-k

        if left < 0:
            return
        
        context_dic = context_list(s, k)

        rna_seq = ""
        for i in range(self.k):
            protein = np.random.choice(np.array(list("ACGT")))
            rna_seq += protein
        
        for i in range(left):
            kmer = rna_seq[-self.k:]
            if kmer in context_dic:
                protein = np.random.choice(np.array(list(context_dic[kmer])))
            else:
                protein = np.random.choice(np.array(list("ACGT")))
            rna_seq += protein
        
        return rna_seq
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

GATGTATGTA


### Idea of solution

fill in

### Discussion

fill in

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [ ]:
def kmer_index(s, k):
    dic = {}
    for i in range(len(s) - k + 1):
        kmer = s[i:i+k]
        if kmer not in dic:
            dic[kmer] = [i]
        else:
            dic[kmer].append(i)
    return dic

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

fill in

### Discussion

fill in

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [ ]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    dic = {"".join(codon): 0 for codon in product("ACGU", repeat=3)}
    for i in range(len(rna)-3):
        three_mer = rna[i:i+3]
        if three_mer not in dic:
            dic[three_mer] = 1
        else:
            dic[three_mer] += 1

    for three_mer in dic:
        dic[three_mer] *= 1/(len(rna) - 2)

    return dic #{"".join(codon): 0 for codon in product("ACGU", repeat=3)}
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    suma = 0
    for event in p:
        if (p[event] == 0.0 and q[event] != 0.0) or (p[event] != 0.0 and q[event] == 0.0):
            return
        elif p[event] == 0 and q[event] == 0:
            continue
        else:
            suma += p[event] * math.log2(p[event] / q[event])
    return suma

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))

    rna = ProteinToRandomRNA().convert(protein)
    #pass
    
    # Maybe check that converting back to protein results in the same sequence
    #DONE
    #pass
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities(rna) # placeholder call
    print(cp_predicted)
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = ProteinToMaxRNA().prob_dic #{"".join(codon): 0 for codon in product("ACGU", repeat=3)} # placeholder dict
    
    # Create a completely random RNA sequence and get the codon probabilities
    random_rna = "".join(choice(list("ACGU"), 3*n))
    pass

    cp_uniform = codon_probabilities(random_rna) # placeholder call
    print(cp_uniform)
    #print(cp_uniform)
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

{'AAA': 0.018967931195413026, 'AAC': 0.015001000066671112, 'AAG': 0.01770118007867191, 'AAU': 0.02093472898193213, 'ACA': 0.01870124674978332, 'ACC': 0.015134342289485965, 'ACG': 0.011534102273484899, 'ACU': 0.016167744516301086, 'AGA': 0.01880125341689446, 'AGC': 0.014300953396893126, 'AGG': 0.014434295619707981, 'AGU': 0.014834322288152544, 'AUA': 0.014834322288152544, 'AUC': 0.0162010800720048, 'AUG': 0.030502033468897925, 'AUU': 0.01806787119141276, 'CAA': 0.01796786452430162, 'CAC': 0.015701046736449096, 'CAG': 0.01900126675111674, 'CAU': 0.021368091206080405, 'CCA': 0.01880125341689446, 'CCC': 0.01360090672711514, 'CCG': 0.008733915594372957, 'CCU': 0.01303420228015201, 'CGA': 0.012134142276151743, 'CGC': 0.009100606707113807, 'CGG': 0.009267284485632375, 'CGU': 0.006967131142076138, 'CUA': 0.012734182278818588, 'CUC': 0.009800653376891793, 'CUG': 0.02123474898326555, 'CUU': 0.01210080672044803, 'GAA': 0.02066804453630242, 'GAC': 0.017367824521634774, 'GAG': 0.017267817854523634,

### Idea of solution

fill in

### Discussion

fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [ ]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    transpose = transition.T
    eigenvalues, eigenvectors = np.linalg.eig(transpose)

    stationary_vectors = eigenvectors.T[eigenvalues == 1.0]
    for vector in stationary_vectors:
        vector *= (1/np.sum(vector))
    
    return stationary_vectors

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    '''print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))'''
    print(get_stationary_distributions(transition))

[[ 0.33333333  0.          0.66666667  0.        ]
 [-0.          0.25       -0.          0.75      ]]


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [ ]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[ 0.33333333  0.          0.66666667  0.        ]
 [-0.          0.25       -0.          0.75      ]]
Using [-0.00, 0.25, -0.00, 0.75] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.29763319
KL divergence of stationary distribution prefix of length    10 is 0.88649534
KL divergence of stationary distribution prefix of length   100 is 0.41770918
KL divergence of stationary distribution prefix of length  1000 is 0.09405583
KL divergence of stationary distribution prefix of length 10000 is 0.34413416


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [ ]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]


IndexError: index 0 is out of bounds for axis 0 with size 0

### Idea of solution

fill in

### Discussion
fill in